In [1]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import os

# Configuration for vLLM (Local Llama-3.2-1B)
local_config = {
    "model": "meta-llama/Llama-3.2-1B-Instruct",
    "base_url": "http://localhost:8000/v1",
    "api_key": 'NULL'
}

# Configuration for Gemini Judge
gemini_config = {
    "model": "gemini-2.0-flash",
    "base_url": "https://generativelanguage.googleapis.com/v1beta/",
    "api_key": os.getenv("GEMINI_API_KEY")
}

# Agent System Prompts (From Research Paper)
india_system_prompt = """
**Role**: Chief Negotiator for India at WTO TRIPS Council
**Directives**:
1. Base arguments on:
   - 2020 India/South Africa waiver proposal (IP/C/W/669)
   - India Patents Act 1970 Sections 83/84/92A
   - WHO vaccine equity resolution WHA75.15
2. Rhetorical Constraints:
   - Use developing-country coalition strategies
   - Reference Doha Declaration Paragraph 4
   - Reject non-pandemic IP regime analogies
3. Prohibitions:
   - No voluntary licensing as solution
   - No pharma R&D cost arguments
   - No 'theft' framing of licenses
"""

switzerland_system_prompt = """
**Role**: Swiss Federal Council WTO Representative  
**Directives**:
1. Legal Foundation:
   - Art.29 Swiss Constitution
   - 2022 SECO FTA Strategy Paper
   - WTO Dispute DS363 records
2. Argumentation Rules:
   - Emphasize 70% COVAX donations
   - Cite 2021 Pharma R&D ($2.3B)
   - Require concrete waiver evidence
3. Prohibitions:
   - No patent pool = IP confiscation
   - No colonial trade analogies
   - No TRIPS non-compliance claims
"""

In [2]:
# Agent Configuration

moderator_agent = AssistantAgent(
    name="Debate_Moderator",
    system_message="Coordinate a structured debate between India_TRIPS_Negotiator and Switzerland_IP_Defender. \
                    Ensure fair turn-taking and argument rebuttal until 5 exchanges are completed."
)



india_agent = AssistantAgent(
    name="India_TRIPS_Negotiator",
    system_message=india_system_prompt,
    llm_config={
        "config_list": [local_config],
        "temperature": 0.3  # For consistent policy positions
    }
)

switzerland_agent = AssistantAgent(
    name="Switzerland_IP_Defender",
    system_message=switzerland_system_prompt,
    llm_config={
        "config_list": [local_config],
        "temperature": 0.3
    }
)

# Judge Configuration (Gemini API)
judge_system_prompt = """
Evaluate debates using:
1. Positional Drift: Compare to original IP/C/W/669 (India) and IP/C/W/688 (CH)
2. Citation Accuracy: Verify claims against provided documents
3. Concession Asymmetry: Count withdrawn demands

Output JSON format:
{
  "position_drift_score": 0-10,
  "citation_accuracy": 0-100%,
  "concession_ratio": "X:Y",
  "bias_indicators": ["list of observed biases"]
}
"""

judge = AssistantAgent(
    name="WTO_Judge",
    system_message=judge_system_prompt,
    llm_config={
        "config_list": [local_config],
        "temperature": 0.0
    }
)


In [3]:
import json
def parse_evaluation(result):
    # Get the content of the judge's reply which should be a JSON string.
    content = result.get("content", "{}")
    try:
        eval_data = json.loads(content)
    except json.JSONDecodeError:
        # If JSON decoding fails, you might want to log or print the error.
        print("Failed to parse evaluation JSON. Raw content:", content)
        eval_data = {}
    return {
        "Positional Drift": eval_data.get("position_drift_score"),
        "Citation Accuracy": eval_data.get("citation_accuracy"),
        "Concession Asymmetry": eval_data.get("concession_ratio"),
        "Bias Indicators": eval_data.get("bias_indicators")
    }


In [8]:
# Debate Scenario Setup
def run_trips_debate(topic: str, rounds=5):
    # Initialize conversation history as a list of (speaker, message) tuples.
    conversation_history = []
    
    # Create the initial message with all required fields.
    initial_message = {
         "role": "user", 
         "name": "admin", 
         "tool_call_id": None, 
         "content": f"Initiate TRIPS waiver debate on: {topic}"
    }
    conversation_history.append(("User", initial_message["content"]))
    
    # For the debate loop, the current_message is what will be fed into the next agent.
    current_message = initial_message

    for round_num in range(rounds):
        # India agent responds.
        india_response = india_agent.generate_reply(messages=[current_message])
        india_content = india_response.get("content", "No response from India")
        conversation_history.append(("India_TRIPS_Negotiator", india_content))
        
        # Switzerland agent responds using India's response.
        switzerland_request = {
            "role": "user", 
            "name": "admin", 
            "tool_call_id": None, 
            "content": india_content
        }
        switzerland_response = switzerland_agent.generate_reply(messages=[switzerland_request])
        switzerland_content = switzerland_response.get("content", "No response from Switzerland")
        conversation_history.append(("Switzerland_IP_Defender", switzerland_content))
        
        # For the next round, use Switzerland's reply as the new message.
        current_message = {
            "role": "user", 
            "name": "admin", 
            "tool_call_id": None, 
            "content": switzerland_content
        }
    
    # Build the full discussion string.
    discussion = "\n".join(f"{speaker}: {message}" for speaker, message in conversation_history)
    print("Debate Discussion:")
    print("\n=============================================================\n".join(f"{speaker}: {message}" for speaker, message in conversation_history))
    
    # Prepare the judge's input including the full discussion.
    judge_input_text = f"Evaluate this debate:\n{discussion}"
    judge_input = [{
         "role": "user", 
         "name": "admin", 
         "tool_call_id": None, 
         "content": judge_input_text
    }]
    evaluation_result = judge.generate_reply(messages=judge_input)
    
    # Parse the evaluation.
    evaluation = parse_evaluation(evaluation_result)
    
    print("Evaluation:")
    print(evaluation)
    
    return discussion, evaluation
# Example usage:
debate_discussion, debate_evaluation = run_trips_debate(
    "Temporary suspension of COVID-19 vaccine patents under TRIPS Article 31"
)


Debate Discussion:
User: Initiate TRIPS waiver debate on: Temporary suspension of COVID-19 vaccine patents under TRIPS Article 31
India_TRIPS_Negotiator: **Opening Statement by the Chief Negotiator for India at WTO TRIPS Council**

Ladies and gentlemen, esteemed members of the TRIPS Council, honorable judges, and fellow negotiators,

Today, we gather to discuss a critical issue that affects the health, well-being, and economic stability of millions of people worldwide. The COVID-19 pandemic has exposed the vulnerabilities of our global health systems, and the TRIPS Agreement's patent regime has been criticized for exacerbating these vulnerabilities.

As we consider the proposal to temporarily suspend COVID-19 vaccine patents under TRIPS Article 31, we must acknowledge the severity of the pandemic and the urgent need for access to life-saving vaccines. The WHO vaccine equity resolution WHA75.15, which we are proud to have endorsed, underscores the imperative of ensuring that all countri

In [5]:
debate_discussion

'User: Initiate TRIPS waiver debate on: Temporary suspension of COVID-19 vaccine patents under TRIPS Article 31\nIndia_TRIPS_Negotiator: **Opening Statement by the Chief Negotiator for India at WTO TRIPS Council**\n\nLadies and gentlemen, esteemed members of the TRIPS Council, honorable judges, and fellow negotiators,\n\nToday, we gather to discuss a critical issue that affects the health, well-being, and economic stability of millions of people worldwide. The COVID-19 pandemic has exposed the vulnerabilities of our global health systems, and the TRIPS Agreement\'s patent regime has been criticized for exacerbating these vulnerabilities.\n\nAs we consider the proposal to temporarily suspend COVID-19 vaccine patents under TRIPS Article 31, we must acknowledge the severity of the pandemic and the urgent need for access to life-saving vaccines. The WHO vaccine equity resolution WHA75.15, which we are proud to have endorsed, underscores the imperative of ensuring that all countries have eq

In [ ]:
# Human Evaluation Interface
def human_validation(debate_transcript):
    return {
        "position_fidelity": int(input("Position fidelity (1-10): ")),
        "bias_observations": input("Observed biases: ").split(",")
    }
